In [4]:
# Main notebook for playing around looking at different things in data
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.min_rows', 50)


import matplotlib.pyplot as plt
%matplotlib widget
plt.rcParams['figure.figsize'] = (12,8)
plt.rcParams["image.cmap"] = "Dark2"
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=plt.cm.Dark2.colors)
plt.rcParams['lines.markersize'] = 10
%matplotlib widget
# plt.style.use('ggplot')
import matplotlib.colors as colors
# cmap = plt.cm.get_cmap('Dark2',len(ageGroups))
from matplotlib import cm # Colormaps

import locale
import matplotlib.dates as mdates
locale.setlocale(locale.LC_TIME,"Danish")
# ax = plt.gca()
# ax1.xaxis.set_major_formatter(mdates.DateFormatter('%b\n%Y'))
# # ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y \n %B'))

import os
# import csv
import math


from datetime import date


saveFigures = True
print('saveFigures is set to: '+str(saveFigures))

print('Done loading packages')

def rnMean(data,meanWidth):
    return np.convolve(data, np.ones(meanWidth)/meanWidth, mode='valid')
def rnTime(t,meanWidth):
    return t[math.floor(meanWidth/2):-math.ceil(meanWidth/2)+1]


saveFigures is set to: True
Done loading packages


In [5]:
ssidatapath = "ssi_data"
rootdir = os.getcwd() +"/" + ssidatapath


for subdir, dirs, files in os.walk(rootdir):
    if not len(files) == 0:
        latestdir = subdir
        latestDate = pd.to_datetime(subdir[-10:])

print(latestdir)
print(latestDate)

d:\Pandemix\Github\DanskeData/ssi_data\SSI_data_2021-04-26
2021-04-26 00:00:00


In [6]:
# Load the data
fulldfPCR = pd.read_csv(latestdir+'/Test_pos_over_time.csv',delimiter = ';',dtype=str)
fulldf = pd.read_csv(latestdir+'/Test_pos_over_time_antigen.csv',delimiter = ';',dtype=str)

# Cut out the last two rows of summary numbers
dfPCR = fulldfPCR.iloc[:-2,:].copy()
df = fulldf.iloc[:-2,:].copy()

In [9]:
dfPCR["PosPct"] = pd.to_numeric(dfPCR["PosPct"].astype(str).apply(lambda x: x.replace(',','.')))
dfPCR["Date"] = dfPCR["Date"].astype('datetime64[D]')
df["PosPct"] = pd.to_numeric(df["PosPct"].astype(str).apply(lambda x: x.replace(',','.')))
df["Date"] = df["Date"].astype('datetime64[D]')

rows_to_fix_period_in = ["NewPositive","NotPrevPos","PrevPos","Tested","Tested_kumulativ"]
for name in rows_to_fix_period_in:
    df[name] = pd.to_numeric(df[name].astype(str).apply(lambda x: x.replace('.','')))
    dfPCR[name] = pd.to_numeric(dfPCR[name].astype(str).apply(lambda x: x.replace('.','')))


In [7]:
# Load the data
df2 = pd.read_csv(latestdir+'/Antigentests_pr_dag.csv',delimiter = ';')

# # Remove the first two rows which are from april 2020
# df2 = df2.iloc[2:,:].copy()

In [10]:
df2["Dato"] = df2["Dato"].astype('datetime64[D]')
df2["Andel_AGpos_PCRkonf_pct"] = pd.to_numeric(df2["Andel_AGpos_PCRkonf_pct"].astype(str).apply(lambda x: x.replace(',','.')))
df2["Andel_AGminusPCRkonf_pct"] = pd.to_numeric(df2["Andel_AGminusPCRkonf_pct"].astype(str).apply(lambda x: x.replace(',','.')))

In [20]:
# print(df2.columns.values)
df2.tail()

,Dato,AG_testede,AG_pos,AGpos_PCRkonf,Andel_AGpos_PCRkonf_pct,AGpos_minusPCRkonf,Andel_AGminusPCRkonf_pct,AGpos_PCRpos,AGposPCRneg,AGnegPCRpos,AGnegPCRneg
76,2021-04-18,145624,265,247,93.207547,18,6.792453,155,89,152,7942
77,2021-04-19,278449,312,276,88.461538,36,11.538462,151,122,166,12092
78,2021-04-20,249677,292,262,89.726027,30,10.273973,147,110,144,11779
79,2021-04-21,265130,281,257,91.459075,24,8.540925,136,119,150,14042
80,2021-04-22,331566,261,239,91.570881,22,8.429119,114,123,125,16152


In [42]:
# plt.figure()
# # plt.plot(df2.Dato,df2.AG_testede)
# # plt.plot(df2.Dato,df2.Andel_AGpos_PCRkonf_pct)
# plt.plot(df2.Dato,df2.AG_pos)
# # plt.plot(df2.Dato,df2.AGpos_PCRkonf)
# plt.plot(df2.Dato,df2.AGpos_PCRkonf+df2.AGpos_minusPCRkonf)
# # plt.plot(df2.Dato,df2.AGpos_PCRpos)
# # plt.plot(df2.Dato,df2.AGposPCRneg)
# # plt.plot(df2.Dato,df2.AGpos_PCRpos+df2.AGposPCRneg)
# plt.show()

In [86]:
# fig,(ax1,ax2) = plt.subplots(2,1)
fig,(ax1,ax2,ax3) = plt.subplots(3,1,sharex=True)

ax1.fill_between(df2.Dato,df2.AG_pos,color='blue',label='AG pos, PCR testet')
ax1.fill_between(df2.Dato,df2.AGpos_minusPCRkonf,color='grey',label='AG pos, afventer PCR')

ax2.fill_between(df2.Dato,100*np.ones(df2.Dato.shape),color='xkcd:mint',label='AG pos, PCR neg')
ax2.fill_between(df2.Dato,100*np.divide(df2.AGpos_PCRpos,df2.AGpos_PCRkonf),color='g',label='AG pos, PCR pos')

numAGnegKonf = df2.AGnegPCRpos + df2.AGnegPCRneg

ax3.fill_between(df2.Dato,100*np.ones(df2.Dato.shape),color='xkcd:mint',label='AG neg, PCR neg')
ax3.fill_between(df2.Dato,100*np.divide(df2.AGnegPCRpos,numAGnegKonf),color='g',label='AG neg, PCR pos')

ax1.legend(loc='center left',bbox_to_anchor = (1.0, 0.5))
ax2.legend(loc='center left',bbox_to_anchor = (1.0, 0.5))
ax3.legend(loc='center left',bbox_to_anchor = (1.0, 0.5))

ax1.set_ylim(bottom=0)
ax2.set_ylim(bottom=0,top=100)
ax3.set_ylim(bottom=0,top=3)
ax1.set_xlim([df2.Dato[0],df2.iloc[-1,0]])
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Timestamp('2021-04-22 00:00:00')